<a href="https://colab.research.google.com/github/renatavel/goit_ds_homeworks/blob/main/hw_13_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras
import matplotlib.pyplot as plt

In [2]:
from keras import datasets
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [5]:
x_train = tf.repeat(x_train, 3, axis=-1)
x_test = tf.repeat(x_test, 3, axis=-1)

In [6]:
x_train = tf.image.resize(x_train, (32, 32))
x_test = tf.image.resize(x_test, (32, 32))

In [7]:
y_train_hot = keras.utils.to_categorical(y_train, 10)
y_test_hot = keras.utils.to_categorical(y_test, 10)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [9]:
from keras import applications
model_base = applications.VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
for layer in model_base.layers:
    layer.trainable = False

In [11]:
for layer in model_base.layers[-4:]:
    layer.trainable = True

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D

model = Sequential([
    model_base,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

In [13]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint_fine_tuning = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

In [15]:
history = model.fit(
    datagen.flow(x_train, y_train_hot, batch_size=64),
    epochs=20,
    validation_data=(x_test, y_test_hot),
    callbacks=[
        EarlyStopping(monitor='val_accuracy', patience=5),
        checkpoint_fine_tuning
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6757 - loss: 0.9436
Epoch 1: val_accuracy improved from -inf to 0.85910, saving model to best_model.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 63s 58ms/step - accuracy: 0.6758 - loss: 0.9433 - val_accuracy: 0.8591 - val_loss: 0.3962
Epoch 2/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8368 - loss: 0.4749
Epoch 2: val_accuracy improved from 0.85910 to 0.86820, saving model to best_model.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.8368 - loss: 0.4748 - val_accuracy: 0.8682 - val_loss: 0.3805
Epoch 3/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8573 - loss: 0.4089
Epoch 3: val_accuracy improved from 0.86820 to 0.87700, saving model to best_model.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 47s 50ms/step - accuracy: 0.8573 - loss: 0.4089 - val_accuracy: 0.8770 - val_loss: 0.3424
Epoch 4/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8658 - loss: 0.3772
Epoch 4: val_accuracy improved from 0.87700 to 0.88040, saving model to best_model.h5


...

938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.8716 - loss: 0.3629 - val_accuracy: 0.8885 - val_loss: 0.3100
Epoch 6/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8765 - loss: 0.3418
Epoch 6: val_accuracy did not improve from 0.88850
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.8765 - loss: 0.3418 - val_accuracy: 0.8834 - val_loss: 0.3272
Epoch 7/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8814 - loss: 0.3233
Epoch 7: val_accuracy improved from 0.88850 to 0.89190, saving model to best_model.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.8814 - loss: 0.3233 - val_accuracy: 0.8919 - val_loss: 0.2959
Epoch 8/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8872 - loss: 0.3085
Epoch 8: val_accuracy did not improve from 0.89190
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.8872 - loss: 0.3085 - val_accuracy: 0.8888 - val_loss: 0.3088
Epoch 9/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8894 - loss: 0.3043
Epoch 9: val_accuracy did not improve from 0.89190
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.8893 - loss: 0.3043 - val_accuracy: 0.8889 - val_loss: 0.3059
Epoch 10/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8937 - loss: 0.2958
Epoch 10: val_accuracy improved from 0.89190 to 0.89430, saving model to best_model.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.8937 - loss: 0.2958 - val_accuracy: 0.8943 - val_loss: 0.2920
Epoch 11/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8919 - loss: 0.2970
Epoch 11: val_accuracy improved from 0.89430 to 0.90040, saving model to best_model.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.8919 - loss: 0.2969 - val_accuracy: 0.9004 - val_loss: 0.2836
Epoch 12/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8969 - loss: 0.2825
Epoch 12: val_accuracy did not improve from 0.90040
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.8969 - loss: 0.2825 - val_accuracy: 0.9001 - val_loss: 0.2881
Epoch 13/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8965 - loss: 0.2771
Epoch 13: val_accuracy did not improve from 0.90040
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.8965 - loss: 0.2771 - val_accuracy: 0.8956 - val_loss: 0.2879
Epoch 14/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9021 - loss: 0.2697
Epoch 14: val_accuracy did not improve from 0.90040
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.9021 - loss: 0.2697 - val_accuracy: 0.8919 - val_loss: 0.2869
Epoch 15/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9018 - loss: 0.2683
Epoch 15: val_ac

938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.9054 - loss: 0.2587 - val_accuracy: 0.9018 - val_loss: 0.2760
Epoch 17/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9031 - loss: 0.2588
Epoch 17: val_accuracy did not improve from 0.90180
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 44ms/step - accuracy: 0.9031 - loss: 0.2588 - val_accuracy: 0.9008 - val_loss: 0.2820
Epoch 18/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9060 - loss: 0.2494
Epoch 18: val_accuracy did not improve from 0.90180
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.9060 - loss: 0.2494 - val_accuracy: 0.8957 - val_loss: 0.2946
Epoch 19/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9095 - loss: 0.2464
Epoch 19: val_accuracy did not improve from 0.90180
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.9095 - loss: 0.2464 - val_accuracy: 0.8961 - val_loss: 0.2865
Epoch 20/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9120 - loss: 0.2419
Epoch 20: val_ac

In [17]:
best_model = keras.models.load_model('best_model.h5')
test_loss, test_accuracy = best_model.evaluate(x_test, y_test_hot)
print(f"Accuracy on the testing data: {test_accuracy * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9022 - loss: 0.2781
Accuracy on the testing data: 90.18%
